# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [ ]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptEncoderOnlyClassifier, VanillaEncoderOnlyClassifier

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [ ]:
KGE_DIMENSIONS = [2, 4, 8, 16] # Output Dimension of the GNN Encoder.

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [ ]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [ ]:
movie_lens_loader.llm_df.head()

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [ ]:
gnn_trainers =    [GNNTrainer(movie_lens_loader.data, kge_dimension = kge_dimension) for kge_dimension in KGE_DIMENSIONS]

We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [ ]:
for gnn_trainer in gnn_trainers:
    print(gnn_trainer.kge_dimension)
    #gnn_trainer.train_model(movie_lens_loader.gnn_train_data, 10)
    gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [ ]:
[gnn_trainer.get_embeddings(movie_lens_loader) for gnn_trainer in gnn_trainers]
movie_lens_loader.llm_df.head()

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [ ]:
vanilla_encoder_only_classifier = VanillaEncoderOnlyClassifier(movie_lens_loader.llm_df)

Next we generate a vanilla llm dataset and tokenize it for training.

In [ ]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_encoder_only_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [ ]:
vanilla_encoder_only_classifier.train_model_on_data(dataset_vanilla, epochs=20)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [ ]:
prompt_encoder_only_classifiers = [PromptEncoderOnlyClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=embedding_dimension) for embedding_dimension, gnn_trainer in zip(KGE_DIMENSIONS, gnn_trainers)]

We also generate a prompt dataset, this time the prompts also include the KGEs.

In [ ]:
datasets_prompt = [movie_lens_loader.generate_prompt_embedding_dataset(prompt_encoder_only_classifier.tokenize_function, kge_dimension = prompt_encoder_only_classifier.kge_dimension) for prompt_encoder_only_classifier in prompt_encoder_only_classifiers]

We also train the model. This can be skipped if already done ones.

In [ ]:
[prompt_encoder_only_classifier.train_model_on_data(dataset_prompt, epochs = 20) for prompt_encoder_only_classifier, dataset_prompt in zip(prompt_encoder_only_classifiers, datasets_prompt)]

In [ ]:
vanilla_encoder_only_classifier.plot_confusion_matrix(dataset=dataset_vanilla, split = "val")

In [ ]:
[prompt_encoder_only_classifier.plot_confusion_matrix(dataset=dataset_prompt, split = "val") for prompt_encoder_only_classifier, dataset_prompt in zip(prompt_encoder_only_classifiers, datasets_prompt)]

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer
import pandas as pd
import numpy as np

In [ ]:
prompt_negative_sample = movie_lens_loader.sample_prompt_datapoint(existing=False, get_embedding_cb=gnn_trainer.get_embedding, tokenize_function=prompt_encoder_only_classifier.tokenize_function)
prompt_positive_sample = movie_lens_loader.sample_prompt_datapoint(tokenize_function=prompt_encoder_only_classifier.tokenize_function)
vanilla_negative_sample = movie_lens_loader.sample_vanilla_datapoint(existing=False, tokenize_function=vanilla_encoder_only_classifier.tokenize_function)
vanilla_positive_sample = movie_lens_loader.sample_vanilla_datapoint(tokenize_function=vanilla_encoder_only_classifier.tokenize_function)

In [ ]:
prompt_negative_sample

# Current State
Here I want to plot the attentions not only between single tokens but between the embedding part and non-embedding part.

In [ ]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind

def foo(self: PromptEncoderOnlyClassifier, sample: dict, layer = -1):
    self.model.eval()
    with torch.no_grad():
        outputs = self.model(input_ids = sample["input_ids"], attention_mask = sample["attention_mask"], output_attentions=True)
        attentions = outputs.attentions  # This will contain the attention weights for each layer and head
    combined_attention = torch.sum(attentions[layer], dim=1).squeeze().detach().numpy()
    # Tokenize the text to get the token labels
    tokens = self.tokenizer.convert_ids_to_tokens(sample['input_ids'][0])
    print(tokens)
    starting_index_user_embeddings = find_sub_list(['user', 'em', '##bed', '##ding', ':', '['], tokens)
    starting_index_movie_embeddings = find_sub_list(['movie', 'em', '##bed', '##ding', ':', '['], tokens)

    # Plot the combined attention weights
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(combined_attention, xticklabels=tokens, yticklabels=tokens, cmap='viridis', ax=ax)
    plt.title('Combined Attention Weights for Layer 1 After Linear Projection')
    plt.xlabel('Tokens')
    plt.ylabel('Tokens')
    plt.show()
foo(prompt_encoder_only_classifiers[0], prompt_negative_sample)